In [1]:
# Batch of imports
import numpy as np
import os
import PIL
import tensorflow as tf
import math

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from som import Kohonen

In [2]:
def condense_color(tensor):
    vect = np.array([])
    for i in tensor:
        for j in i:
            avg = np.average(j.numpy(), returned=True)[0]
            if avg > 0.5:
                vect = np.append(vect, int(0))
            else:
                vect = np.append(vect, int(1))
    return vect

def condense_gray(tensor):
    threshold = 0.5
    l = tensor.numpy().shape[0]
    w = tensor.numpy().shape[1]
    vect = tensor.numpy().reshape(-1,) < threshold
    # imgPrint(vect, l, w)
    return vect

def imgPrint(a, l, w):
    for i in range(l):
        line = ""
        for j in range(w):
            line += str(int(a[i * l + j])) + " "
        print(line)


In [3]:
input_dir = "./CharacterSets/Training"  # Training set directory
val_split = 0.2                         # Validation split
img_height = img_width = 50             # Height and width of the input images
batch_size = 100                        # Batch size

# Defining the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    input_dir,
    labels='inferred',
    validation_split=val_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size)  # WTF does batch size affect????

# Defining the validation dataset
"""
val_ds = tf.keras.utils.image_dataset_from_directory(
    input_dir,
    labels='inferred',
    validation_split=val_split,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
"""


Found 731402 files belonging to 62 classes.
Using 585122 files for training.


'\nval_ds = tf.keras.utils.image_dataset_from_directory(\n    input_dir,\n    labels=\'inferred\',\n    validation_split=val_split,\n    subset="validation",\n    seed=123,\n    image_size=(img_height, img_width),\n    batch_size=batch_size)\n'

Notes:
train_ds upon initialization is a BatchDataset object
mnist in NN.py is a Module object
x_train is a complex numpy array

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
# print(class_names)

# Normalization layer. first_image can be viewed as a numpy array, but currently consists of all 1s for some reason.
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
arr = condense_gray(first_image)

# Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))
print(type(first_image))
testSet = np.array([])
for i in range(len(image_batch)):
    testSet = np.append(testSet, condense_gray(image_batch[i]))
testSet = testSet.reshape(-1, 2500)
imgPrint(testSet[0], 50, 50)